In [ ]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark import StorageLevel
from geopyspark import geopyspark_conf
from geopyspark.geotrellis import Extent
from geopyspark.geotrellis.layer import TiledRasterLayer, Pyramid
from geopyspark.geotrellis.color import ColorMap
from geopyspark.geotrellis.constants import *
import json
import shapely
from shapely.geometry import shape, asShape, MultiPoint, MultiLineString

conf = geopyspark_conf("yarn-client", "MVP")
conf.set("spark.hadoop.yarn.timeline-service.enabled", False)
pysc = SparkContext.getOrCreate(conf)

## Rasterize Libya Roads to Raster RDD

In [ ]:
import requests
libya_roads_json = requests.get('https://s3.amazonaws.com/geopyspark-demo/libya/roads.geojson').json()
libya_roads = MultiLineString([shape(geom['geometry']) for geom in libya_roads_json['features']])

In [ ]:
# Try to be friedly with python libraries like shapely
from geopyspark import geotrellis
Extent(*libya_roads.bounds)

In [ ]:
# All execution time here is sending WKB over py4j socket
from geopyspark.geotrellis.rasterize import rasterize
from geopyspark.geotrellis import RasterizerOptions

ro = RasterizerOptions(includePartial=True, sampleType='PixelIsArea')

road_raster = rasterize(pysc, geoms=libya_roads.geoms, 
                        crs="EPSG:3857", zoom=8, 
                        fill_value=1, cell_type="float32",
                        options = ro,
                        numPartitions = 20)

road_raster.layer_metadata.bounds

## Show Rasterized Roads on a Map

In [ ]:
# Pyramid up from base layer
road_pp = road_raster.pyramid(start_zoom=8, end_zoom=1).cache()

In [ ]:
from geopyspark.geotrellis.tms import *
from geonotebook.wrappers.raster import TMSRasterData

# color map roads 1 to red
roads_cm = ColorMap.from_colors(pysc, [1], [0xff000080])

# start JVM tile server and serve tiles to map
server = TMSServer.rdd_tms_server(pysc, road_pp, roads_cm)
M.add_layer(TMSRasterData(server), name="TMS")

In [ ]:
M.remove_layer(M.layers[0])

## Cost Distance Based on Road Network

In [ ]:
# road network will shape our friction layer

road_friction = (
    road_raster
    .reclassify({1:1}, data_type=int, replace_nodata_with=10)
)

In [ ]:
# starting points for cost distance operation

population_json = requests.get('https://s3.amazonaws.com/geopyspark-demo/libya/population.geojson').json()
population_centers = MultiPoint([shape(geom['geometry']) for geom in population_json['features']])

conflict_json = requests.get('https://s3.amazonaws.com/geopyspark-demo/libya/conflict.geojson').json()
conflict_centers = MultiPoint([shape(feature['geometry']) for feature in conflict_json['features'] if feature['geometry'] != None])

conflict_centers

In [ ]:
from geonotebook.wrappers import VectorData
M.add_layer(VectorData("population.geojson"), name="Population")

In [ ]:
# wait for the iterations to settle
from geopyspark.geotrellis.cost_distance import cost_distance

pop_cd = cost_distance(
    road_friction,
    population_centers, 
    max_distance=1400000.0
)

pop_pp = pop_cd.pyramid(start_zoom=8, end_zoom=1)

In [ ]:
con_cd = cost_distance(
    road_friction,
    geometries = conflict_centers, 
    max_distance=1400000.0
)

con_pp = con_cd.pyramid(start_zoom=8, end_zoom=1)

In [ ]:
# prepare color map for weighted overlay based on max cost
from geopyspark.geotrellis.color import get_colors_from_matplotlib
breaks = [x for x in range(0, 1000000, 10000)]
colors = get_colors_from_matplotlib('viridis', len(breaks))
wo_cm = ColorMap.from_colors(pysc, breaks=breaks, color_list=colors)

In [ ]:
from geopyspark.geotrellis.tms import *
from geonotebook.wrappers.raster import TMSRasterData

In [ ]:
weighted_overlay = (con_pp * 0.0) + (pop_pp * 1.0)

server = TMSServer.rdd_tms_server(pysc, weighted_overlay, wo_cm)
M.add_layer(TMSRasterData(server), name="WO")

In [ ]:
# remove the next to last layer
M.remove_layer(M.layers[-2])